In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models, applications, optimizers, callbacks
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
import numpy as np
import cv2
import os
from imblearn.over_sampling import RandomOverSampler
from imblearn.tensorflow import balanced_batch_generator

2025-01-28 21:42:23.832894: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-01-28 21:42:25.049546: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/cuda/lib64:/usr/local/cuda/lib64:/usr/local/cuda-11.8/lib64:/usr/local/cuda/lib64:/usr/local/cuda/lib64:/usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/local/cuda/extras/CUPTI/lib64:/usr/local/cuda-12.6/lib64
2025-01-28 21:42:25.049643: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7

In [2]:
# Configuration
IMG_SIZE = 224  # Increased resolution for transfer learning
BATCH_SIZE = 32
EPOCHS = 50
CLASSES = ['anger', 'contempt', 'disgust', 'fear', 'happiness', 'neutral', 'sadness', 'surprise']

In [ ]:
# 1. Data Loading & Stratified Split
# --------------------------------------------------
def load_dataset(dataset_path):
    images = []
    labels = []
    
    for class_idx, class_name in enumerate(CLASSES):
        class_dir = os.path.join(dataset_path, class_name)
        if not os.path.exists(class_dir): continue
            
        for img_file in os.listdir(class_dir):
            img_path = os.path.join(class_dir, img_file)
            try:
                img = cv2.imread(img_path)
                img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Keep color information
                img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
                images.append(img)
                labels.append(class_idx)
            except Exception as e:
                print(f"Error loading {img_path}: {str(e)}")
    
    return np.array(images), np.array(labels)

# Load data
X, y = load_dataset("/home/natalyagrokh/img_datasets/combo_ferckja_dataset_2")

# Stratified split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, stratify=y_train, random_state=42)

Error loading /home/natalyagrokh/img_datasets/combo_ferckja_dataset_2/anger/._KL.AN3.169.tiff: OpenCV(4.10.0) /home/conda/feedstock_root/build_artifacts/libopencv_1735819704602/work/modules/imgproc/src/color.cpp:196: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'

Error loading /home/natalyagrokh/img_datasets/combo_ferckja_dataset_2/anger/._NM.AN2.105.tiff: OpenCV(4.10.0) /home/conda/feedstock_root/build_artifacts/libopencv_1735819704602/work/modules/imgproc/src/color.cpp:196: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'

Error loading /home/natalyagrokh/img_datasets/combo_ferckja_dataset_2/anger/._UY.AN3.148.tiff: OpenCV(4.10.0) /home/conda/feedstock_root/build_artifacts/libopencv_1735819704602/work/modules/imgproc/src/color.cpp:196: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'

Error loading /home/natalyagrokh/img_datasets/combo_ferckja_dataset_2/anger/._TM.AN1.190.tiff: OpenCV(4.10.0) /home/conda/feedstock_root/build

In [ ]:
# 2. Handle Class Imbalance
# --------------------------------------------------
# Oversample minority classes
ros = RandomOverSampler(random_state=42)
X_train_reshaped = X_train.reshape(X_train.shape[0], -1)
X_train_balanced, y_train_balanced = ros.fit_resample(X_train_reshaped, y_train)
X_train_balanced = X_train_balanced.reshape(-1, IMG_SIZE, IMG_SIZE, 3)

# Calculate class weights
class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
class_weights = dict(enumerate(class_weights))

In [ ]:
# 3. Data Augmentation (Focus on minority classes)
# --------------------------------------------------
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rotation_range=25,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    preprocessing_function=applications.efficientnet.preprocess_input
)

In [ ]:
# 4. Transfer Learning Model
# --------------------------------------------------
base_model = applications.EfficientNetB3(
    weights='imagenet',
    include_top=False,
    input_shape=(IMG_SIZE, IMG_SIZE, 3)
)

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(512, activation='relu'),
    layers.Dropout(0.5),
    layers.BatchNormalization(),
    layers.Dense(len(CLASSES), activation='softmax')
])

# Freeze base layers
base_model.trainable = False

# Custom learning rate
optimizer = optimizers.Adam(learning_rate=1e-3)

model.compile(
    optimizer=optimizer,
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
# 5. Training with Callbacks
# --------------------------------------------------
early_stop = callbacks.EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True
)

lr_scheduler = callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=3
)

checkpoint = callbacks.ModelCheckpoint(
    'best_model.h5',
    monitor='val_accuracy',
    save_best_only=True
)

In [ ]:
# 6. Train with Balanced Batches
# --------------------------------------------------
train_generator = balanced_batch_generator(
    X_train_balanced,
    y_train_balanced,
    batch_size=BATCH_SIZE,
    sampler=RandomOverSampler()
)

history = model.fit(
    train_generator,
    steps_per_epoch=len(X_train_balanced) // BATCH_SIZE,
    validation_data=(applications.efficientnet.preprocess_input(X_val), y_val),
    epochs=EPOCHS,
    callbacks=[early_stop, lr_scheduler, checkpoint],
    class_weight=class_weights
)

In [ ]:
# 7. Fine-tuning
# --------------------------------------------------
# Unfreeze some layers
base_model.trainable = True
for layer in base_model.layers[:100]:
    layer.trainable = False

model.compile(
    optimizer=optimizers.Adam(learning_rate=1e-5),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

history_fine = model.fit(
    train_generator,
    steps_per_epoch=len(X_train_balanced) // BATCH_SIZE,
    validation_data=(applications.efficientnet.preprocess_input(X_val), y_val),
    epochs=int(EPOCHS * 0.5),
    callbacks=[early_stop, lr_scheduler]
)

In [ ]:
# 8. Evaluation
# --------------------------------------------------
test_preprocessed = applications.efficientnet.preprocess_input(X_test)
test_loss, test_acc = model.evaluate(test_preprocessed, y_test)
print(f"Test Accuracy: {test_acc:.4f}")